In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score

In [47]:
df=pd.read_csv("Churn_Modelling.csv")
df.shape

(10000, 14)

In [48]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [49]:
df.drop(columns=['RowNumber','CustomerId','Surname'],inplace=True)

In [50]:
df.shape

(10000, 11)

In [51]:
df.isnull().mean()*100

,0
CreditScore,0.0
Geography,0.0
Gender,0.0
Age,0.0
Tenure,0.0
Balance,0.0
NumOfProducts,0.0
HasCrCard,0.0
IsActiveMember,0.0
EstimatedSalary,0.0


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [53]:
df.duplicated().sum()

np.int64(0)

In [54]:
X=df.drop(columns=['Exited'])
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [55]:
df['Exited'].value_counts()

,count
Exited,
0,7963
1,2037


In [56]:
Y=df['Exited']
Y

,Exited
0,1
1,0
2,1
3,0
4,0
...,...
9995,0
9996,0
9997,1
9998,1


In [57]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=10,test_size=0.2)

In [58]:
num_col=[cols for cols in df.columns if df.dtypes[cols] in ['int64','float64'] and cols!='Exited']
num_col

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [59]:
cat_col=[cols for cols in df.columns if df.dtypes[cols] in ['object'] ]
cat_col

['Geography', 'Gender']

In [60]:
preprocessor=ColumnTransformer(transformers=[
    ("ohe",OneHotEncoder(drop='first',sparse_output=False),cat_col),
    ("scaler",StandardScaler(),num_col)],remainder="passthrough"
)

In [61]:
pipe1=Pipeline(
    [
        ("preprocessor",preprocessor),
        ("model",RandomForestClassifier())
    ]
)

In [62]:
pipe1.fit(X_train,Y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['Geography', 'Gender']),
                                                 ('scaler', StandardScaler(),
                                                  ['CreditScore', 'Age',
                                                   'Tenure', 'Balance',
                                                   'NumOfProducts', 'HasCrCard',
                                                   'IsActiveMember',
                                                   'EstimatedSalary'])])),
                ('model', RandomForestClassifier())])

In [63]:
y_pred_test=pipe1.predict(X_test)
y_pred_train=pipe1.predict(X_train)

In [64]:
print(f"The training accuracy is:{accuracy_score(Y_train,y_pred_train):.3f}")
print(f"The testing accuracy is:{accuracy_score(Y_test,y_pred_test):.3f}")
print(f"The training f1 score is:{f1_score(Y_train,y_pred_train):.3f}")
print(f"The testing f1 score is:{f1_score(Y_test,y_pred_test):.3f}")

The training accuracy is:1.000
The testing accuracy is:0.855
The training f1 score is:1.000
The testing f1 score is:0.574


In [65]:
pipe2=Pipeline(
    [
        ("preprocessor",preprocessor),
        ("model",LogisticRegression())
    ]
)

In [66]:
pipe2.fit(X_train,Y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['Geography', 'Gender']),
                                                 ('scaler', StandardScaler(),
                                                  ['CreditScore', 'Age',
                                                   'Tenure', 'Balance',
                                                   'NumOfProducts', 'HasCrCard',
                                                   'IsActiveMember',
                                                   'EstimatedSalary'])])),
                ('model', LogisticRegression())])

In [67]:
y_pred_test=pipe2.predict(X_test)
y_pred_train=pipe2.predict(X_train)

In [68]:
print(f"The training accuracy is:{accuracy_score(Y_train,y_pred_train):.3f}")
print(f"The testing accuracy is:{accuracy_score(Y_test,y_pred_test):.3f}")
print(f"The training f1 score is:{f1_score(Y_train,y_pred_train):.3f}")
print(f"The testing f1 score is:{f1_score(Y_test,y_pred_test):.3f}")

The training accuracy is:0.813
The testing accuracy is:0.799
The training f1 score is:0.320
The testing f1 score is:0.295


In [69]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [70]:
model=Sequential()

In [71]:
model.add(Dense(3,activation='sigmoid',input_dim=11))
model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [72]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=["accuracy"])

In [73]:
X_train_preprocessed=preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [74]:
model.fit(X_train_preprocessed,Y_train,epochs=100,validation_split=0.2)

Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7934 - loss: 0.5260 - val_accuracy: 0.8112 - val_loss: 0.4761
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7979 - loss: 0.4856 - val_accuracy: 0.8112 - val_loss: 0.4544
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7880 - loss: 0.4778 - val_accuracy: 0.8112 - val_loss: 0.4411
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7952 - loss: 0.4570 - val_accuracy: 0.8112 - val_loss: 0.4323
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7918 - loss: 0.4512 - val_accuracy: 0.8112 - val_loss: 0.4254
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7953 - loss: 0.4404 - val_accuracy: 0.8112 - val_loss: 0.4205
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7966 - loss: 0.4354 - val_accuracy: 0.8112 - val_loss: 0.4170
Epoch 8/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7900 - loss: 0.4380 - val_accu

In [75]:
model.layers[0].get_weights()

[array([[-3.87776703e-01, -2.05639434e+00, -1.63856828e+00],
        [-5.33877015e-01,  1.27763140e+00, -1.58929583e-02],
        [ 4.46910203e-01,  8.16214979e-01,  1.26233542e+00],
        [ 8.74527991e-02,  9.74830762e-02, -4.34588920e-03],
        [-3.48892832e+00, -2.19089365e+00,  2.10804105e-01],
        [-2.62197386e-03,  6.02627024e-02,  1.90249216e-02],
        [ 3.44675004e-01, -7.36414731e-01, -4.80430037e-01],
        [ 1.10869944e-01, -9.87730026e-02,  2.73200661e-01],
        [-2.66856521e-01,  4.84604895e-01,  1.31290287e-01],
        [-1.02645898e+00,  9.87691209e-02,  2.10839176e+00],
        [ 2.44926691e-01, -4.47544195e-02, -4.37497646e-01]], dtype=float32),
 array([ 0.24025904,  0.01136527, -0.4021779 ], dtype=float32)]

In [76]:
model.layers[1].get_weights()

[array([[-2.24956  ],
        [-1.0167284],
        [-2.2425032]], dtype=float32),
 array([1.1966246], dtype=float32)]

In [77]:
y_log_train=model.predict(X_train_preprocessed)
y_log_test=model.predict(X_test_preprocessed)

250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [78]:
y_pred_test=np.where(y_log_test>0.5,1,0)
y_pred_train=np.where(y_log_train>0.5,1,0)

In [79]:
print(f"The training accuracy is:{accuracy_score(Y_train,y_pred_train):.3f}")
print(f"The testing accuracy is:{accuracy_score(Y_test,y_pred_test):.3f}")
print(f"The training f1 score is:{f1_score(Y_train,y_pred_train):.3f}")
print(f"The testing f1 score is:{f1_score(Y_test,y_pred_test):.3f}")

The training accuracy is:0.838
The testing accuracy is:0.823
The training f1 score is:0.454
The testing f1 score is:0.419
